# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x20268472fa0>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-11-30")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98817,Zimbabwe,NaN,Zimbabwe__nan,2020-11-26,115.0,2.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
98818,Zimbabwe,NaN,Zimbabwe__nan,2020-11-27,91.0,2.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
98819,Zimbabwe,NaN,Zimbabwe__nan,2020-11-28,108.0,2.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
98820,Zimbabwe,NaN,Zimbabwe__nan,2020-11-29,0.0,2.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 243.10365933046535
Test MAE: 239.27522830880685


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -14 NewCases 0.01063511120413984
Day -7 NewCases 0.3901833563851552
Day -6 NewCases 0.13934255461612743
Day -4 NewCases 0.12938927512555035
Day -3 NewCases 0.049278982260770494
Day -2 NewCases 0.07721298168596523
Day -1 NewCases 0.23360962065089283
Day -27 C6_Stay at home requirements 10.081487437570491
Day -18 C6_Stay at home requirements 13.859149414673897
Day -10 C6_Stay at home requirements 5.464966906524101
Intercept 41.675232903469805


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 110.721549344085
2020-08-02: 132.85696554341925
2020-08-03: 146.01904406625135
2020-08-04: 163.24031979966742
2020-08-05: 181.43132439511896
2020-08-06: 180.02027812762324
2020-08-07: 199.12830368230138
2020-08-08: 252.81951918007303
2020-08-09: 279.5077050098621
2020-08-10: 298.20358136686855
2020-08-11: 313.70944547675066
2020-08-12: 333.9813118403191
2020-08-13: 346.11359395449927
2020-08-14: 368.63332678789396
2020-08-15: 403.68194682213084
2020-08-16: 430.0831156889156
2020-08-17: 451.23185456554194
2020-08-18: 471.9099025627045
2020-08-19: 480.14399095447106
2020-08-20: 495.97914841569474
2020-08-21: 517.9433903826953
2020-08-22: 545.3036200405745
2020-08-23: 568.7689485036241
2020-08-24: 590.8265656062034
2020-08-25: 611.3619824372208
2020-08-26: 628.1937646381347
2020-08-27: 647.202767061426
2020-08-28: 659.3496068988902
2020-08-29: 681.4594541200113
2020-08-30: 703.1871419561869
2020-08-31: 724.7211427531294

Predicting for Afghanistan__n

2020-08-16: 2422.3112015648176
2020-08-17: 2408.4080135947115
2020-08-18: 2461.6842200495657
2020-08-19: 2429.2630726086672
2020-08-20: 2276.678155516797
2020-08-21: 2304.4621318180925
2020-08-22: 2471.8732079693864
2020-08-23: 2549.351959356418
2020-08-24: 2563.2383608290884
2020-08-25: 2601.333162716952
2020-08-26: 2602.54560002639
2020-08-27: 2556.497135682995
2020-08-28: 2563.8961157749754
2020-08-29: 2647.0259103968424
2020-08-30: 2698.3795155804637
2020-08-31: 2721.7799557000394

Predicting for Azerbaijan__nan
2020-08-01: 3185.769085093367
2020-08-02: 3588.899516198826
2020-08-03: 3027.3519190987067
2020-08-04: 3523.815374522946
2020-08-05: 3384.1000884396094
2020-08-06: 1821.5485328918448
2020-08-07: 1843.1766909520234
2020-08-08: 3082.9375224666246
2020-08-09: 3360.14247791073
2020-08-10: 3155.3457993155203
2020-08-11: 3384.855573276566
2020-08-12: 3322.4014491864614
2020-08-13: 2701.2806392258544
2020-08-14: 2717.360476261753
2020-08-15: 3256.0182162712053
2020-08-16: 3428.317

2020-08-02: 135.97231024969113
2020-08-03: 150.478713922087
2020-08-04: 163.4350266088478
2020-08-05: 178.26569370900071
2020-08-06: 179.75912375417295
2020-08-07: 200.2975329063928
2020-08-08: 255.45402758172654
2020-08-09: 281.6917512227832
2020-08-10: 300.7933424607827
2020-08-11: 308.9447194124899
2020-08-12: 326.66317442134164
2020-08-13: 339.04237927970473
2020-08-14: 361.8750002665186
2020-08-15: 396.50048465855394
2020-08-16: 422.36948327620456
2020-08-17: 442.5564056758106
2020-08-18: 459.7175506151978
2020-08-19: 452.1180391331872
2020-08-20: 464.03734351135506
2020-08-21: 483.64482216502415
2020-08-22: 508.6931502822019
2020-08-23: 528.848299661458
2020-08-24: 548.4702857684773
2020-08-25: 564.1055118347124
2020-08-26: 572.3914345494195
2020-08-27: 586.6236844824919
2020-08-28: 585.1182808242447
2020-08-29: 601.2470275225327
2020-08-30: 617.5455827220599
2020-08-31: 634.4152612897261

Predicting for Bosnia and Herzegovina__nan
2020-08-01: 678.4613086439142
2020-08-02: 747.92

2020-08-22: 1065.8499905894205
2020-08-23: 1110.0001970437338
2020-08-24: 1150.025259256789
2020-08-25: 1187.612547026443
2020-08-26: 1229.8656350849694
2020-08-27: 1247.8651659019388
2020-08-28: 1256.8485277517414
2020-08-29: 1284.2016251911302
2020-08-30: 1321.0757388239294
2020-08-31: 1356.007777084476

Predicting for Brazil__Amapa
2020-08-01: 157.28648304646535
2020-08-02: 281.63338635204855
2020-08-03: 324.92301964415606
2020-08-04: 305.9826422294909
2020-08-05: 542.8539452225042
2020-08-06: 402.5134908035451
2020-08-07: 315.4696035812825
2020-08-08: 372.21868723874115
2020-08-09: 457.03785268199874
2020-08-10: 476.1008137269047
2020-08-11: 495.72539061827433
2020-08-12: 593.6380042547906
2020-08-13: 558.2036606719265
2020-08-14: 532.2478527044564
2020-08-15: 566.4468050231177
2020-08-16: 620.4283091844645
2020-08-17: 640.4485900061849
2020-08-18: 668.7196234652154
2020-08-19: 705.8813291788484
2020-08-20: 705.7815826601939
2020-08-21: 706.3233480867412
2020-08-22: 733.40078304720

2020-08-26: 1389.3002380950625
2020-08-27: 1408.290127486399
2020-08-28: 1431.8439196006248
2020-08-29: 1466.6719475273262
2020-08-30: 1510.8870743013597
2020-08-31: 1554.5092869437005

Predicting for Brazil__Para
2020-08-01: 370.5374565159466
2020-08-02: 445.30703782600045
2020-08-03: 688.6340624811442
2020-08-04: 839.5598060253935
2020-08-05: 994.7090339374656
2020-08-06: 937.3677824767525
2020-08-07: 578.375173647121
2020-08-08: 672.253459005228
2020-08-09: 746.7940229400803
2020-08-10: 867.0191577626947
2020-08-11: 934.0490489834845
2020-08-12: 1028.4103900305495
2020-08-13: 999.5256451060958
2020-08-14: 880.0197936923419
2020-08-15: 914.1519038147853
2020-08-16: 970.3161723312573
2020-08-17: 1035.2833977189462
2020-08-18: 1081.9238077137543
2020-08-19: 1111.744897441125
2020-08-20: 1104.2487808248125
2020-08-21: 1069.8135801475448
2020-08-22: 1090.83697020349
2020-08-23: 1128.3383291827101
2020-08-24: 1169.1821689327335
2020-08-25: 1201.2662690840543
2020-08-26: 1228.0780364011234

...

2020-08-23: 872.9374650169266
2020-08-24: 895.8233267442629
2020-08-25: 922.154396913822
2020-08-26: 939.1531515610279
2020-08-27: 947.5360355001517
2020-08-28: 959.3569311207107
2020-08-29: 990.4766713116618
2020-08-30: 1017.9247502322798
2020-08-31: 1041.2177611207157

Predicting for United States__Michigan
2020-08-01: 2971.914391999333
2020-08-02: 2457.5676365554627
2020-08-03: 4733.330658658621
2020-08-04: 4364.032361831235
2020-08-05: 3859.9055104248673
2020-08-06: 1957.4165367274975
2020-08-07: 2157.237163821299
2020-08-08: 3063.6690374204472
2020-08-09: 3197.1019613074977
2020-08-10: 3979.5028402845855
2020-08-11: 3999.583015861498
2020-08-12: 3718.7994358459705
2020-08-13: 2946.7113757673983
2020-08-14: 3051.1590685336214
2020-08-15: 3408.5794187872084
2020-08-16: 3581.3777796732984
2020-08-17: 3886.870608514001
2020-08-18: 3967.4889919163697
2020-08-19: 3828.309524585554
2020-08-20: 3532.4981939598993
2020-08-21: 3588.830377727817
2020-08-22: 3755.899815359098
2020-08-23: 3878

2020-08-31: 4490.234935393901

Predicting for United States__New Mexico
2020-08-01: 1272.9210869765277
2020-08-02: 1415.3750941745752
2020-08-03: 1309.3661684831807
2020-08-04: 1320.04782186478
2020-08-05: 1478.680238648587
2020-08-06: 814.444921040033
2020-08-07: 836.2673422958281
2020-08-08: 1315.3635037231097
2020-08-09: 1453.8411475563107
2020-08-10: 1399.0454527660054
2020-08-11: 1441.613902438938
2020-08-12: 1491.2450182999019
2020-08-13: 1246.067433362967
2020-08-14: 1262.9047404512517
2020-08-15: 1475.6297109818488
2020-08-16: 1568.8753155495253
2020-08-17: 1559.6137021803468
2020-08-18: 1600.9501359743288
2020-08-19: 1615.0404743664378
2020-08-20: 1532.7501244942423
2020-08-21: 1551.896289758282
2020-08-22: 1657.1480483154742
2020-08-23: 1717.5472294250962
2020-08-24: 1731.643314722292
2020-08-25: 1765.80879713565
2020-08-26: 1786.0325426087122
2020-08-27: 1769.8567296711324
2020-08-28: 1783.3811054319067
2020-08-29: 1842.454813902321
2020-08-30: 1885.6414831922852
2020-08-31:

2020-08-02: 649.0950355711063
2020-08-03: 473.22741838913385
2020-08-04: 555.7431837818744
2020-08-05: 736.3127134529133
2020-08-06: 430.19998234658283
2020-08-07: 436.4906978530381
2020-08-08: 666.0999715353231
2020-08-09: 732.1848880590306
2020-08-10: 665.5813530210413
2020-08-11: 719.4543515243914
2020-08-12: 793.6082420080808
2020-08-13: 692.1818550898597
2020-08-14: 702.6992169727961
2020-08-15: 812.8524735376889
2020-08-16: 861.2969068364391
2020-08-17: 848.1628908336204
2020-08-18: 887.8549759354942
2020-08-19: 915.6150843634407
2020-08-20: 889.4209012318486
2020-08-21: 905.2213517262048
2020-08-22: 965.5659366153536
2020-08-23: 1000.9589543919426
2020-08-24: 1010.9736590156577
2020-08-25: 1040.9924201032659
2020-08-26: 1066.3006334476868
2020-08-27: 1070.506164922647
2020-08-28: 1080.8218368132186
2020-08-29: 1118.3363656861807
2020-08-30: 1147.0989329501817
2020-08-31: 1165.7180306885614

Predicting for United States__Tennessee
2020-08-01: 5975.14499110981
2020-08-02: 8872.577

2020-08-25: 3109.838244431633
2020-08-26: 3110.1158297521633
2020-08-27: 3050.171680587778
2020-08-28: 3082.9635264223125
2020-08-29: 3191.426299254079
2020-08-30: 3241.5733581907407
2020-08-31: 3271.868037508511

Predicting for United States__West Virginia
2020-08-01: 1080.8971871692318
2020-08-02: 1095.7242633521378
2020-08-03: 1094.2877026407891
2020-08-04: 1175.5949350585179
2020-08-05: 1113.5722591954836
2020-08-06: 660.2122682237382
2020-08-07: 700.8550770710224
2020-08-08: 1112.7056397478932
2020-08-09: 1182.5104323823703
2020-08-10: 1185.9101618536092
2020-08-11: 1236.7229017617349
2020-08-12: 1216.3722754267753
2020-08-13: 1041.3761912116988
2020-08-14: 1075.1144355572224
2020-08-15: 1256.9752847945924
2020-08-16: 1318.6760603948346
2020-08-17: 1334.543560315262
2020-08-18: 1374.1970349535459
2020-08-19: 1364.4133954049455
2020-08-20: 1305.554038711722
2020-08-21: 1333.9929674540228
2020-08-22: 1424.6771716967057
2020-08-23: 1470.9193169311689
2020-08-24: 1494.262404039875
202

2020-08-04: 209.28770530674117
2020-08-05: 202.7109735535185
2020-08-06: 195.0468731835658
2020-08-07: 216.94904791968804
2020-08-08: 284.0962337694572
2020-08-09: 311.03313289242465
2020-08-10: 332.6666954078679
2020-08-11: 344.3914620309513
2020-08-12: 354.3528155903365
2020-08-13: 361.9003310617842
2020-08-14: 385.71190909882614
2020-08-15: 425.3691133566746
2020-08-16: 451.88607616250647
2020-08-17: 473.5503793355641
2020-08-18: 491.92068486847575
2020-08-19: 481.4730776213809
2020-08-20: 491.13082000051764
2020-08-21: 511.33841872463654
2020-08-22: 538.5495245355944
2020-08-23: 559.2424187931683
2020-08-24: 579.6658379412672
2020-08-25: 595.8269727417796
2020-08-26: 603.0950645616655
2020-08-27: 616.3890314699066
2020-08-28: 615.2613943486921
2020-08-29: 632.39615609875
2020-08-30: 649.1093445393745
2020-08-31: 666.4524744422345

Predicting for Zimbabwe__nan
2020-08-01: 159.63976858767103
2020-08-02: 194.36665462609244
2020-08-03: 226.05818673158612
2020-08-04: 272.56751539968207


In [22]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,110.721549
214,Aruba,NaN,2020-08-02,132.856966
215,Aruba,NaN,2020-08-03,146.019044
216,Aruba,NaN,2020-08-04,163.240320
217,Aruba,NaN,2020-08-05,181.431324


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [23]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [24]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,110.721549344085
Aruba,,2020-08-02,132.85696554341925
Aruba,,2020-08-03,146.01904406625135
Aruba,,2020-08-04,163.24031979966742
Afghanistan,,2020-08-01,229.17688977298798
Afghanistan,,2020-08-02,326.44440936029133
Afghanistan,,2020-08-03,303.8069782507652
Afghanistan,,2020-08-04,330.51054499811056
Angola,,2020-08-01,156.9399774033422


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [25]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [26]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [27]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
Wall time: 1.92 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [28]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-24
End date: 2021-06-22


In [29]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [30]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-24 to 2021-06-22...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
Wall time: 4min 38s
